In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime
import json

%matplotlib inline
#from finrl.config_tickers import DOW_30_TICKER
#from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
#from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
#from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
#from finrl.agents.stablebaselines3.models import DRLAgent,DRLEnsembleAgent
#from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline

from pprint import pprint

import sys, os
sys.path.append(os.path.abspath(".."))

import itertools


In [2]:
from finHRL.preprocess.yahoodownloader import YahooDownloader

with open("../finHRL/preprocess/tickers/ticker_lists.json", "r") as f:
    data = json.load(f)

dow_30 = data["DOW_30"]
cryptos = data["CRYPTO_7"]

TRAIN_START_DATE = '2014-01-01'
TRAIN_END_DATE = '2022-01-01'
TEST_START_DATE = '2022-01-01'
TEST_END_DATE = '2025-01-01'



df = YahooDownloader(start_date = pd.to_datetime(TRAIN_START_DATE) - datetime.timedelta(days=30),
                     end_date = TEST_END_DATE,
                     ticker_list = dow_30).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Shape of DataFrame:  (83670, 8)


In [3]:
df

Price,date,close,high,low,open,volume,tic,day
0,2013-12-02,17.097761,17.504089,17.085044,17.307749,472544800,AAPL,0
1,2013-12-02,82.360474,82.648272,81.864021,82.230965,2185500,AMGN,0
2,2013-12-02,19.615000,19.950001,19.455000,19.950001,94280000,AMZN,0
3,2013-12-02,72.077675,73.125584,71.967816,72.694587,3620800,AXP,0
4,2013-12-02,114.662712,115.773787,114.201182,115.372091,3378100,BA,0
...,...,...,...,...,...,...,...,...
83665,2024-12-31,237.962219,238.811767,235.917386,237.497930,795100,TRV,1
83666,2024-12-31,497.021118,502.071322,494.545161,500.450158,4214000,UNH,1
83667,2024-12-31,313.811127,315.419697,312.967115,313.890549,3963800,V,1
83668,2024-12-31,37.414078,37.451498,37.002418,37.086621,13549900,VZ,1


In [4]:
INDICATORS = ['macd',
              'rsi_30',
              'cci_30',
              'dx_30']

from finHRL.preprocess.preprocessor import FeatureEngineer
fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list = INDICATORS,
                     use_turbulence=True,
                     user_defined_feature = False)

processed = fe.preprocess_data(df)
processed = processed.copy()
processed = processed.fillna(0)
processed = processed.replace(np.inf,0)

processed = processed[processed.date >= TRAIN_START_DATE].reset_index(drop=True)

Successfully added technical indicators
Successfully added turbulence index


In [5]:
processed["dayorder"] = processed["date"].astype("category").cat.codes

In [6]:
stock_dimension = len(processed.tic.unique())
print(stock_dimension)

30


In [7]:
# state_space_manager = [close prices_i, MACD_i, rsi30_i, cci30_i, dx30_i, turbulences_i] Quizas quitar algún indicador
# state_space_worker = [balance, close_prices_i, stock_shares_i, manager_actions_i]
# state_space_noHRL = [balance, close prices_i, stock_shares_i, MACD_i, rsi30_i, cci30_i, dx30_i, turbulences_i]
episode_len = processed.dayorder.nunique()
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension # TODO se tendrá que actualizar

env_kwargs = {
    "hmax": 100, #
    "initial_amount": 1000000,
    "buy_cost_pct": 0.001,
    "sell_cost_pct": 0.001,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4,
    "print_verbosity":5

}

from finHRL.env_stocktrading.trading_env_finrl import StockTradingEnv

buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension


tr_env = StockTradingEnv(
    df = processed,
    stock_dim=stock_dimension,
    hmax= 1000,
    initial_amount=1000000,
    num_stock_shares=num_stock_shares,
    buy_cost_pct=buy_cost_list,
    sell_cost_pct=sell_cost_list,
    reward_scaling=1e-4,
    state_space= state_space,
    action_space= stock_dimension,
    tech_indicator_list=INDICATORS,
    make_plots=True,
    print_verbosity=1
)



In [8]:
from finHRL.agent.models import baseRLAgent

agent = baseRLAgent(env=tr_env)
model_ppo = agent.get_model("ppo",
                            verbose=1)

n_episodes = 100


train_ppo = agent.train_model(
    model_ppo,
    tb_log_name="ppo_test1",
    total_timesteps= n_episodes*episode_len
)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
-----------------------------
| time/              |      |
|    fps             | 77   |
|    iterations      | 1    |
|    time_elapsed    | 26   |
|    total_timesteps | 2048 |
-----------------------------
day: 2767, episode: 1
begin_total_asset: 1000000.00
end_total_asset: 1105830.63
total_reward: 105830.63
total_cost: 1482207.92
total_trades: 57664
Sharpe: 0.141
--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 2.77e+03 |
|    ep_rew_mean          | 0.0978   |
| time/                   |          |
|    fps                  | 74       |
|    iterations           | 2        |
|    time_elapsed         | 55       |
|    total_timesteps      | 4096     |
| train/                  |          |
|    approx_kl            | 0.038078 |
|    clip_fraction        | 0.365    |
|    clip_range           | 0.2      |
|    entropy_loss         | 

# Pendiente

- LR está en 0.0003: Probar a bajar a 5e-5 o 1e-5